In [1]:
import requests
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from CertificationAuthority import CertificationAuthority
import datetime
import util
import os
from cryptography import exceptions, x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.primitives import hashes

In [2]:
# create 2 root server
ca1=CertificationAuthority('ca1')
ca2=CertificationAuthority('ca2')

In [3]:
# create client 1 's certificate sign request and rsa key 
rsa_key=util.generate_ras_key()
rsa_key.public_key()
csr = x509.CertificateSigningRequestBuilder().subject_name(x509.Name([
    # Provide various details about who we are.
    x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
    x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"California"),
    x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
    x509.NameAttribute(NameOID.COMMON_NAME, u"client1.com"),
])).add_extension(
    x509.SubjectAlternativeName([
        # Describe what sites we want this certificate for.
        x509.DNSName(u"mysite.com"),
        x509.DNSName(u"www.mysite.com"),
        x509.DNSName(u"subdomain.mysite.com"),
    ]),
    critical=False,
# Sign the CSR with our private key. is applicant private key
).sign(rsa_key, hashes.SHA256())

In [4]:
# get cert from ca1
cert_pub=ca1.issue_certificate(csr)
print(cert_pub)

b'-----BEGIN CERTIFICATE-----\nMIIDQjCCAiqgAwIBAgIUbKMUPDEIJM4FelJCHHjG5oy39NkwDQYJKoZIhvcNAQEL\nBQAwUTELMAkGA1UEBhMCU0cxEjAQBgNVBAgMCVNpbmdhcG9yZTESMBAGA1UEBwwJ\nU2luZ2Fwb3JlMQwwCgYDVQQKDANjYTExDDAKBgNVBAMMA2NhMTAeFw0yMTExMjQw\nNjU4NThaFw0yMTEyMDQwNjU4NThaMGUxCzAJBgNVBAYTAlVTMRMwEQYDVQQIDApD\nYWxpZm9ybmlhMRYwFAYDVQQHDA1TYW4gRnJhbmNpc2NvMRMwEQYDVQQKDApNeSBD\nb21wYW55MRQwEgYDVQQDDAtjbGllbnQxLmNvbTCCASIwDQYJKoZIhvcNAQEBBQAD\nggEPADCCAQoCggEBAM8KEMtMEbktOmP36f38VjarW6k95IV5qW6wobGrgTZTV3Ur\nE0QwZnQXx3tp869dSfgWad6APd45Zrx35dOyQ1Am1TR5ugU8QGqoW3ikLrkjbHMG\nurn5DC7qw99Rnn3AkMo3JljKNTD1zE5j6KKeHUl27W3YH6d4zhjBZ8vm9iJKkReU\n5a90XXzua8rGZ20yU5M/eFPvO6OzJPnOTjhJBPtlS/9iNAKA6eWHuuOPmwrTVaNt\nHMLDfR1AseAB+fK9tqHVI3l+m7WjmyXx4aKgjEzc13BpdqOIqOzy8GkmGNnkAsm1\ndZvW1UCyKJ/1Lxjd2Bvkx0I7gV85u5gdSsLQbXUCAwEAATANBgkqhkiG9w0BAQsF\nAAOCAQEAZDIYZnx6wfbUAZpv0hJPWKuzo9uLSqz62GDQzqcN7e0UQ5IxRee+2Yes\n1v5JdvQOKke0fINpFCfSaLTHl8uTq+zfasfjsuUv12e7OoWNgxKLi81EUCcQo0rG\nCRwBwfPm2ahvT4Mpcr0+tvQXTeX12v4CDIOV67cm1xO4t

In [7]:
#reconstruct from cert public byte
cert = x509.load_pem_x509_certificate(cert_pub)
# info of cert
print("======cert info========")
print("applicant/subject: "+cert.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print("issuer: "+cert.issuer.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value)
print(cert.not_valid_before)
print(cert.not_valid_after)

# verify signature
print('=====verify signature========')
ca_public_key=serialization.load_pem_public_key(ca1.get_public_key())
print("validate: "+ str(util.verify_cert_signature(cert,ca_public_key)))

======cert info========
applicant/subject: client1.com
issuer: ca1
2021-11-24 06:58:58
2021-12-04 06:58:58
=====verify signature========
validate: True


In [ ]:
# ca1 issue cert revoke from ca2

# verify chain